In [ ]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio
from dash.dependencies import Input, Output
import json


In [ ]:
# Read price data from Excel
price_data = pd.read_excel('data/price_data_frq.xlsx', sheet_name= None, index_col=0)
schedule_data = pd.read_excel('data/schedule_data.xlsx', index_col=0)

# Parameters
Seff = 1
Ceff = 1
Deff = 1
cap_power = 1  # Example value
cap_energy = 4  # Example value
η = 1000
dt = 1

λ_0 = 0.5
λ_min = 0.0
initial_soc = λ_0 * cap_energy


In [ ]:
data = price_data

# Iteratively merge all dataframes on 'Timestamp'
price_df = pd.DataFrame(index =  data[list(data.keys())[0]].index)

for key in data:
    price_df = pd.concat([price_df, data[key]], axis = 1)


# Model

In [ ]:
s

# Run

In [ ]:
time_period = 24    # periods

total_time_period = len(price_df)

num_slices = total_time_period // time_period

# Lists to store results
total_revenue = 0
arbitrage = 0
regulation = 0

all_charging_schedules = []
all_discharging_schedules = []
all_soc_schedules = []

first_p = 0
last_p = num_slices - 1

# Run the optimization for each time period
for p in range(num_slices):
    print("watch", p)
    periodic_price = price_df[p*time_period:(p+1)*time_period]
    revenue, charging_schedule, discharging_schedule, soc_schedule, [rev_arbitrage, rev_regulation] = optimize_revenue(initial_soc, periodic_price, p, first_p, last_p)
    final_soc = soc_schedule[-1]
    
    # Store the results
    total_revenue += revenue
    arbitrage += rev_arbitrage
    regulation += rev_regulation

    all_charging_schedules.extend(charging_schedule)
    all_discharging_schedules.extend(discharging_schedule)
    all_soc_schedules.extend(soc_schedule[:-1])

    # Update the initial SOC for the next day
    initial_soc = final_soc

print(total_revenue)

In [ ]:
arbitrage

In [ ]:
regulation/arbitrage

In [ ]:
27168/185

In [ ]:
# Create a DataFrame for the results
data = {
    't': range(1, len(all_charging_schedules) + 1),
    'charge': all_charging_schedules,
    'discharge': all_discharging_schedules,
    'soc': all_soc_schedules
}

result_df = pd.DataFrame(data)

result_df = result_df.set_index(['t'])

result_df['net_discharge'] = result_df['discharge'] - result_df['charge']
result_df['soc_percent'] = result_df['soc'] /cap_energy

result_df

In [ ]:
# Create interactive plot for charging and discharging
fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=result_df.index, y=result_df['net_discharge'], mode='lines', name='net'))
fig_charge_discharge.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (MW)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_charge_discharge.show()

# Create interactive plot for charging and discharging
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=result_df.index, y=result_df['soc_percent']*100, mode='lines', name='net', line=dict(color='orange')))
fig_soc.update_layout(
    title='State of Charge',
    xaxis_title='Time',
    yaxis_title='SOC (%)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_soc.show()

In [ ]:
# Create the combined layout
combined_html = f"""
<html>
    <head>
        <title>BESS Revenue Simulation Dashboard</title>
        <style>
            .responsive-plot {{
                width: 80%;
                max-width: 3600px;
                height: 80%;
                max-height: 600px;
                margin: auto;
            }}
        </style>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

    </head>
    <body>
        <h1>BESS Revenue Simulation Dashboard</h1>
        <div>
            <h3>Charging and Discharging Schedules</h3>
            <div id="plot1" class="responsive-plot">
                {pio.to_html(fig_charge_discharge, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
        <div>
            <h3>State of Charge Over Time</h3>
            <div id="plot2" class="responsive-plot">
                {pio.to_html(fig_soc, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
    </body>
</html>
"""

# Save the combined HTML to a file
with open("bess_dashboard.html", "w") as f:
    f.write(combined_html)